In [1]:
import torch
import os

from script.xai.plot_and_print import plot_data_scatter
from script.data_processing.process_csv import merge_data, process_spatial_data, ann_data_to_csv
import pandas as pd
from model import load_model

In [2]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
data_dir_train = "../Training_Data/"
data_dir_test = "../Test_Data/"
patients_train = [os.path.basename(f) for f in os.scandir(data_dir_train) if f.is_dir()]
patients_test = [os.path.basename(f) for f in os.scandir(data_dir_test) if f.is_dir()][1:]
print(patients_train)
print(patients_test)

model_dir = "../remote_models/"
skip = 0
model_dir_path = []

for model_type_dir in os.listdir(model_dir):
    sub_path = model_dir + model_type_dir
    if model_type_dir == ".DS_Store" or os.path.isfile(sub_path):
        continue
    for model_leaf_dir in os.listdir(sub_path):
        sub_path = model_dir + model_type_dir + "/" + model_leaf_dir
        if model_type_dir == ".DS_Store" or os.path.isfile(sub_path):
            continue
        files = os.listdir(sub_path)
        for f in files:
            if f[-3:] == ".pt" and f.find("ep_") != -1:
                src = sub_path + "/" + f
                dst = sub_path + "/" + f[f.find("ep_"):]
                os.rename(src, dst)
                if f[f.find("ep_"):] == "ep_29.pt":
                    model_dir_path.append((sub_path + "/", dst))
print(model_dir_path)
models = [load_model(x, y, log_json=True) for x, y in model_dir_path]

In [3]:
for patient in patients:
    ann_data_to_csv(data_dir)
    process_spatial_data(patients, data_dir)
    merge_data(data_dir, patient)

In [16]:
def process_and_plot_patient(model, device, data_dir, patient, gene = "RUBCNL", results_filename = "results.csv"):
    generate_results(model, device, data_dir, patient, gene, results_filename)
    merge_data(data_dir, patient, results_filename)
    plot_data_scatter(data_dir, patient, gene)

In [10]:
columns = ["model_name", "model_type", "gene"]
to_append = [[s + "_mean_out", s + "_std_out", s + "_mean_labels", s + "_std_labels", s + "_pearson", s] for s in patients_test]
for s in to_append:
    for t in s:
        columns.append(t)
print(len(columns))
# mean_out, std_out, mean_labels, std_labels, pearson, patient
print(patients_test)

In [11]:
print(len(lines[0]))
df = pd.DataFrame(lines, columns=columns)
df.drop(patients_test, axis=1, inplace=True)
df.head()


In [12]:
df.to_csv("results_all_models_test.csv")

In [19]:
data_dir = "../Training_Data/"

patients = [os.path.basename(f) for f in os.scandir(data_dir) if f.is_dir()]
patient = "p026"
for model in models:
    print(model[1])
    process_and_plot_patient(model[0].to(device), device, data_dir_test, patient, "RUBCNL", results_filename = "slim_model_results.csv")
    

In [20]:
from process_csv import generate_results
from model import load_model, generate_model_list
import pandas as pd

import torchmetrics
MSE = torchmetrics.MeanSquaredError()
import scipy

pearson = scipy.stats.pearsonr(df["labels"], df["output"])
print(pearson)
model_list_file_name = "new_models.csv"
update_model_list = True

must_contain = None
skip_names = ["AE", "dropout"]
if not os.path.exists(model_dir + model_list_file_name) or update_model_list:
    print("found these models:")
    frame = generate_model_list(model_dir, must_contain=must_contain, skip_names=skip_names)
else:
    frame = pd.read_csv(model_dir + model_list_file_name)

data_dir = "../data/jonas/Training_Data/"
device="cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
out_file = "../results/val_results.csv"
patients = ["p009", "p013"]


for idx, row in frame.iterrows():
    for patient in patients:

        model = load_model(row["model_dir"], row["mode_path"], log_json=True).to(device)
        results_filename="val_results.csv"
        
        file_path = generate_results(model, device=device, data_dir=data_dir, patient=patient, genes=["RUBCNL"], results_filename=results_filename)
        df = pd.read_csv(file_path)
        df.head()
        pearson = scipy.stats.pearsonr(df["labels"], df["output"])
        mse = MSE(df["output"], df["labels"])
        #print(pearson)
        if os.path.exists(out_file):
            results_df = pd.read_csv(out_file)
        else:
            results_df = pd.DataFrame(columns=["model_path", "pearson_p009", "mse_p009", "pearson_p013", "mse_p013"])
        
        results_df.at[row["model_path"], "pearson_" + patient] = pearson
        results_df.at[row["model_path"], "mse_" + patient] = mse
        results_df.to_csv(out_file)
    

{'model_type': 'resnet18', 'random_weights': False, 'dropout': False, 'pretrained_out_dim': '1000', 'loss_fn': 'MSELoss()', 'learning_rate': 0.0005, 'batch_size': 128, 'genes': ['RUBCNL'], 'epochs': 40, 'optimizer': 'AdamW (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: None\n    initial_lr: 0.0005\n    lr: 0.0005\n    maximize: False\n    weight_decay: 0.005\n\nParameter Group 1\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: None\n    initial_lr: 0.0005\n    lr: 0.0005\n    maximize: False\n    weight_decay: 0.005\n)', 'scheduler': '<torch.optim.lr_scheduler.ExponentialLR object at 0x148c60e809e0>', 'device': 'cuda', 'freeze_pretrained': False}
<All keys matched successfully>
generating results...


,labels,output,path,tile
0,0.297868,0.583432,../data/jonas/Training_Data//p013/tiles/p013_1...,p013_1_0.tiff
1,1.550453,0.204171,../data/jonas/Training_Data//p013/tiles/p013_2...,p013_2_0.tiff
2,0.974872,-0.103596,../data/jonas/Training_Data//p013/tiles/p013_3...,p013_3_0.tiff
3,0.813911,0.755017,../data/jonas/Training_Data//p013/tiles/p013_4...,p013_4_0.tiff
4,0.165409,0.469784,../data/jonas/Training_Data//p013/tiles/p013_5...,p013_5_0.tiff


PearsonRResult(statistic=0.6282925947048605, pvalue=3.3811790146560316e-102)
